# Longitudinal Analysis
This notebook recreates the longitudinal analysis of a DDoS2Vec model as described in the paper.

**Warning**: Unlike previous notebooks, this notebook is documented less thoroughly. This is because it is already specific to our dataset in terms of the data's time ranges.

In [ ]:
import collections as cll
import datetime as dt
import gc
import gzip
import itertools as it
import multiprocessing as mp
import multiprocessing.pool as mp_pool
import os
import pickle
import random
import typing as T

import numpy as np
import numpy.typing as T_np
import joblib

import sklearn.metrics as metrics

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

Global notebook options and variables.

In [ ]:
ARTEFACTS_PATH = "./Artefacts/"  # Where the monthly corpora are located.

SEED = 463  # Same as in `2_Baseline_Comparison.ipynb`.

CLASSIFIER_NAME: T.Literal[
    "k-NN",
    "XGBoost",
    "Dummy",
] = "k-NN"

# By having the training month as 6 (June), we can test a few months into the
# past up to 1 (January) and test a few months into the future up to 12
# (December). This is the best analysis we can do with the least amount of
# processing/compute, instead of training on one month and testing on every
# other month with all months getting a chance to be the training month.
TRAINING_MONTH: int = 6
MAX_CONCURRENT_MONTHS: T.Optional[int] = 2

# We used the untuned hyperparameter configuration from the previous
# experiments in the paper.
SVD_COMPONENTS = 100
NGRAM_RANGE = (1, 3)

BINARISER_TRAIN_PATH = os.path.join(
    ARTEFACTS_PATH, "binariser_train.pickle_joblib.gz"
)
TF_IDF_PATH = os.path.join(ARTEFACTS_PATH, "tf_idf.pickle_joblib.gz")
SVD_PATH = os.path.join(ARTEFACTS_PATH, "svd.pickle_joblib.gz")
CLASSIFIER_PATH = os.path.join(
    ARTEFACTS_PATH, f"{CLASSIFIER_NAME}.pickle_joblib.gz"
)

binariser_train: MultiLabelBinarizer
tf_idf: TfidfVectorizer
svd: TruncatedSVD

Helper functions.

In [ ]:
# This is a hack to make the default multiprocessing pool not spawn daemon
# processes; hence, we can then use pools within child processes made by a
# pool. Will cause a lot of zombie processes or fail to exit cleanly if
# something ancestral crashes, as they're no longer marked daemonic.
class NoDaemonProcess(mp.Process):
    @property
    def daemon(self):
        return False

    @daemon.setter
    def daemon(self, _):
        pass


class CustomPool(mp_pool.Pool):  # See above.
    def Process(self, *args, **kwargs):
        proc = super().Process(*args, **kwargs)  # type: ignore
        proc.__class__ = NoDaemonProcess
        return proc


LOG_LOCK = mp.Lock()


def log(*args, **kwargs):
    with LOG_LOCK:
        print(
            f"[{dt.datetime.now().strftime('%d-%m %I:%M:%S %p')}]:",
            *args,
            **kwargs,
        )


def load_pickle(path: str, compressed=True) -> T.Any:
    try:
        gc.disable()
        if compressed:
            assert path.endswith(".gz")
            with gzip.open(path, "rb") as file:
                return pickle.load(file)
        else:
            with open(path, "rb") as file:
                return pickle.load(file)
    except Exception as exception:
        raise exception from None
    finally:
        gc.enable()


def save_pickle(object: T.Any, path: str, overwrite=True, compress=True):
    if overwrite or not os.path.isfile(path):
        if compress:
            assert path.endswith(".gz")
            with gzip.open(path, "wb", compresslevel=6) as file:
                pickle.dump(object, file, protocol=pickle.HIGHEST_PROTOCOL)
        else:
            with open(path, "wb") as file:
                pickle.dump(object, file, protocol=pickle.HIGHEST_PROTOCOL)
        return True
    return False


def build_classifier():
    log(f'Using classifier "{CLASSIFIER_NAME}".')

    if CLASSIFIER_NAME == "k-NN":
        from sklearn.neighbors import KNeighborsClassifier

        return KNeighborsClassifier(
            n_neighbors=10, n_jobs=-1, weights="distance"
        )
    elif CLASSIFIER_NAME == "XGBoost":
        from xgboost import XGBClassifier

        return XGBClassifier(tree_method="hist", n_jobs=-1, random_state=SEED)
    elif CLASSIFIER_NAME == "Dummy":
        from sklearn.dummy import DummyClassifier

        return DummyClassifier(random_state=SEED)

    raise ValueError(f"Unknown classifier: {CLASSIFIER_NAME}")


# Using a lambda function here causes a pickling error when trying to save the
# TF-IDF vectoriser. That is likely a Jupyter Notebook issue.
def return_argument(argument: T.Any) -> T.Any:
    return argument

Load the corpus, etc.

In [ ]:
def load_corpus(
    month: T.Union[int, str], binariser: T.Optional[MultiLabelBinarizer] = None
):
    corpus: T.Dict[str, T.List[str]] = load_pickle(
        os.path.join(ARTEFACTS_PATH, f"{month}.standard.data.pickle.gz")
    )
    labels: T.Dict[str, T.Dict[T.Optional[str], int]] = load_pickle(
        os.path.join(ARTEFACTS_PATH, f"{month}.standard.labels.pickle.gz")
    )

    x: T.List[T.Tuple[str, ...]] = []
    y_tags: T.List[str] = []

    while corpus:
        tag, words = corpus.popitem()
        x.append(tuple(words))
        y_tags.append(tag)

    label_generator = (
        (label if label is not None else "-" for label in labels[tag])
        for tag in y_tags
    )
    if binariser is not None:
        binariser_classes = frozenset(binariser.classes_).union((None,))
        for tag_labels in labels.values():
            for tag_label in frozenset(tag_labels):
                if tag_label not in binariser_classes:
                    if None in tag_labels:
                        tag_labels[None] += tag_labels[tag_label]
                    else:
                        tag_labels[None] = tag_labels[tag_label]
                    del tag_labels[tag_label]
        y: T_np.NDArray[T.Any] = binariser.transform(
            label_generator
        )  # type: ignore
    else:
        binariser = MultiLabelBinarizer(sparse_output=False)
        y: T_np.NDArray[T.Any] = binariser.fit_transform(
            label_generator
        )  # type: ignore

    return x, y, y_tags, binariser

In [ ]:
log("Loading corpus...")
x_train, y_train, y_tags_train, binariser_train = load_corpus(TRAINING_MONTH)
joblib.dump(
    binariser_train,
    BINARISER_TRAIN_PATH,
    compress=6,
    protocol=pickle.HIGHEST_PROTOCOL,
)
log("Loaded corpus.")

random.seed(SEED)
np.random.seed(SEED)

Train the model on the chosen month.

In [ ]:
tf_idf = TfidfVectorizer(
    ngram_range=NGRAM_RANGE,
    lowercase=False,
    tokenizer=return_argument,
    preprocessor=return_argument,
    token_pattern=None,  # type: ignore
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
)

log("Fitting TF-IDF vectoriser...")
tf_idf_matrix_train = tf_idf.fit_transform(x_train)
log("Fitted TF-IDF vectoriser.")
joblib.dump(tf_idf, TF_IDF_PATH, compress=6, protocol=pickle.HIGHEST_PROTOCOL)
log("Saved TF-IDF vectoriser.")

The low dimensional SVD is just for visualization purposes.
The below code cell can be removed if time is an issue.

In [ ]:
low_dimensional_svd = TruncatedSVD(
    n_components=2,
    algorithm="arpack",
    random_state=SEED,
)

log("Fitting low dimensional SVD...")
low_dimensional_decomposition = low_dimensional_svd.fit_transform(
    tf_idf_matrix_train
)
log("Fitted low dimensional SVD.")
joblib.dump(
    low_dimensional_decomposition,
    os.path.join(
        ARTEFACTS_PATH, "low_dimensional_decomposition.pickle_joblib.gz"
    ),
    compress=6,
)
log("Saved low-dimensional transformed TF-IDF matrix.")
del low_dimensional_svd, low_dimensional_decomposition  # Save memory.

In [ ]:
svd = TruncatedSVD(
    n_components=SVD_COMPONENTS,
    algorithm="arpack",
    random_state=SEED,
)

log("Fitting SVD...")
decomposition_train = svd.fit_transform(tf_idf_matrix_train)
log("Fitted SVD.")
joblib.dump(svd, SVD_PATH, compress=6, protocol=pickle.HIGHEST_PROTOCOL)
joblib.dump(
    decomposition_train,
    os.path.join(ARTEFACTS_PATH, "decomposition_train.pickle_joblib.gz"),
    compress=8,
    protocol=pickle.HIGHEST_PROTOCOL,
)
log("Saved SVD and transformed TF-IDF matrix.")

classifier = build_classifier()
log(f"Fitting {CLASSIFIER_NAME}...")
classifier = classifier.fit(decomposition_train, y_train)
log(f"Fitted {CLASSIFIER_NAME}.")
joblib.dump(
    classifier, CLASSIFIER_PATH, compress=6, protocol=pickle.HIGHEST_PROTOCOL
)
log(f"Saved {CLASSIFIER_NAME}.")

The training of the model is done at this point. 

The below cell just loads the persisted model (if not already in the namespace).

In [ ]:
try:
    binariser_train  # type: ignore
except NameError:
    binariser_train: MultiLabelBinarizer = joblib.load(BINARISER_TRAIN_PATH)

try:
    tf_idf  # type: ignore
except NameError:
    tf_idf: TfidfVectorizer = joblib.load(TF_IDF_PATH)

try:
    svd  # type: ignore
except NameError:
    svd: TruncatedSVD = joblib.load(SVD_PATH)

try:
    classifier  # type: ignore
except NameError:
    classifier = joblib.load(CLASSIFIER_PATH)

In [ ]:
def test(month: T.Union[int, str]):
    scores_file = os.path.join(ARTEFACTS_PATH, f"{month}.scores.pickle.gz")
    if os.path.isfile(scores_file):
        log(f"[{month}] Scores already computed.")
        return load_pickle(scores_file)

    log(f"[{month}] Loading corpus...")
    x_test, y_test, _, _ = load_corpus(month, binariser_train)

    start_time = dt.datetime.now(tz=dt.timezone.utc)

    log(f"[{month}] Computing TF-IDF...")
    tf_idf_matrix_test = tf_idf.transform(x_test)

    log(f"[{month}] Computing the SVD of the TF-IDF matrix...")
    decomposition_test = svd.transform(tf_idf_matrix_test)

    log(f"[{month}] Predicting...")
    y_predictions = classifier.predict(decomposition_test)

    end_time = dt.datetime.now(tz=dt.timezone.utc)

    log(f"[{month}] Scoring individual class labels...")

    individual_scores: T.Dict[str, T.Dict[str, float]] = {}
    for index, scores in enumerate(
        zip(
            *np.asarray(
                metrics.precision_recall_fscore_support(
                    y_test, y_predictions, average=None, zero_division=0
                )
            ),
            np.asarray(
                metrics.jaccard_score(
                    y_test, y_predictions, average=None, zero_division=0
                )
            ),
        )
    ):
        individual_scores[binariser_train.classes_[index]] = {
            "Precision": scores[0],
            "Recall": scores[1],
            "F1 Score": scores[2],
            "Support": scores[3],
            "Jaccard Score": scores[4],
        }

    for index, confusion_matrix in enumerate(
        metrics.multilabel_confusion_matrix(y_test, y_predictions)
    ):
        individual_scores[binariser_train.classes_[index]].update(
            {
                "False Negatives": confusion_matrix[1][0],
                "True Negatives": confusion_matrix[0][0],
                "False Positives": confusion_matrix[0][1],
                "True Positives": confusion_matrix[1][1],
            }
        )

    log(f"[{month}] Scoring over samples...")

    precision, recall, f1_score, _ = metrics.precision_recall_fscore_support(
        y_test, y_predictions, average="samples", zero_division=0
    )

    scores = {
        "Start Time": start_time,
        "End Time": end_time,
        "Classes": individual_scores,
        "Average (Samples)": {
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1_score,
            "Jaccard Score": metrics.jaccard_score(
                y_test, y_predictions, average="samples", zero_division=0
            ),
            "Exact Match Ratio": metrics.accuracy_score(y_test, y_predictions),
            "Hamming Loss": metrics.hamming_loss(y_test, y_predictions),
        },
    }

    save_pickle(scores, scores_file)

    return scores

In [ ]:
scores_file_path = os.path.join(ARTEFACTS_PATH, "scores.pickle.gz")
if not os.path.isfile(scores_file_path):
    if MAX_CONCURRENT_MONTHS and MAX_CONCURRENT_MONTHS > 1:
        with CustomPool(MAX_CONCURRENT_MONTHS, maxtasksperchild=1) as pool:
            scores = pool.map(test, range(1, 13))
    else:
        scores = list(map(test, range(1, 13)))

    save_pickle(scores, scores_file_path)
else:
    log("Scores already exist. Loading...")
    scores = load_pickle(scores_file_path)

# Visualisations, etc.

**Warning**: The following code is extremely messy and not well documented. 
It is not recommended to use this code for anything other than the visualisations in the paper.
We provide this purely as an example of what you could do with a model.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats("svg", "pdf")


# An example of recalculating some metrics in case the testing set's individual
# sample predictions are lost.
def macro_and_micro(month_scores):
    precisions, recalls, f1_scores = [], [], []
    true_positives, false_positives, false_negatives = 0, 0, 0
    for class_scores in month_scores["Classes"].values():
        precisions.append(class_scores["Precision"])
        recalls.append(class_scores["Recall"])
        f1_scores.append(class_scores["F1 Score"])
        true_positives += int(class_scores["True Positives"])
        false_positives += int(class_scores["False Positives"])
        false_negatives += int(class_scores["False Negatives"])

    macro_precision = float(np.mean(precisions))
    macro_recall = float(np.mean(recalls))
    macro_f1_score = float(np.mean(f1_scores))
    micro_precision = true_positives / (true_positives + false_positives)
    micro_recall = true_positives / (true_positives + false_negatives)
    micro_f1_score = (
        2
        * true_positives
        / (2 * true_positives + false_positives + false_negatives)
    )

    return (
        (
            (macro_precision + micro_precision) / 2,
            (macro_recall + micro_recall) / 2,
            (macro_f1_score + micro_f1_score) / 2,
        ),
        (macro_precision, macro_recall, macro_f1_score),
        (micro_precision, micro_recall, micro_f1_score),
    )


metric = "Exact Match Ratio"

x_axis = list(range(1, 13))
y_axis_one_label = "Exact Match Ratio (Subset Accuracy)"
y_axis_one = [score["Average (Samples)"][metric] for score in scores]

# Some months have next to no support (number of samples) for a filtering
# rule, so because they are extremely insignificant, they are hard to predict.
y_axis_two_label = (
    "Macro-Averaged F1 Scores of Rules with "
    "at least 1% of the Month's Majority Label Support"
)
y_axis_two = [
    np.mean(
        [
            label["F1 Score"]
            for label in score["Classes"].values()
            if label["Support"]
            >= (
                max(
                    majority["Support"]
                    for majority in score["Classes"].values()
                )
                * 0.01  # Must have 1% of the majority label's support.
            )
        ]
    )
    for score in scores
]

top_classes = list(
    class_label
    for class_label, _ in sorted(
        scores[TRAINING_MONTH - 1]["Classes"].items(),
        key=lambda x: x[1]["Support"],
        reverse=True,
    )[:15]
)
metric = "Precision"
y_axis_three_label = f"Macro-Averaged {metric} of Top 15 Rules by Support"
y_axis_three = [
    np.mean([score["Classes"][top_class][metric] for top_class in top_classes])
    for score in scores
]

y_axis_bar_colours = ["g" for _ in range(len(y_axis_three))]
y_axis_bar_colours[TRAINING_MONTH - 1] = "purple"

plt.gca().set_ylim((0.0, 1.0))
plt.gca().set_xlabel("Month of 2019")
plt.gca().set_ylabel("Score/Metric Value")
plt.gca().set_xticks(x_axis)
plt.gca().set_yticks(np.arange(0.0, 1.1, 0.1))
plt.gcf().set_size_inches(10, 4)
plt.plot(x_axis, y_axis_one, "-o", label=y_axis_one_label, color="orange")
plt.bar(x_axis, y_axis_two, color=y_axis_bar_colours, label=y_axis_two_label)
plt.plot(x_axis, y_axis_three, "-o", label=y_axis_three_label, color="cyan")
plt.legend(loc="lower center")
plt.tight_layout()
plt.savefig(
    os.path.join(ARTEFACTS_PATH, "figure.longitudinal_analysis_overview.pdf")
)
plt.show()

In [ ]:
vocabularies_file_path = os.path.join(
    ARTEFACTS_PATH, "month_vocabularies.pickle.gz"
)


def load_vocabulary(month: T.Union[int, str]):
    vocabulary: T.DefaultDict[str, int] = cll.defaultdict(int)
    log(f"Loading corpus for month {month}...")
    corpus: T.Dict[str, T.List[str]] = load_pickle(
        os.path.join(ARTEFACTS_PATH, f"{month}.standard.data.pickle.gz")
    )
    log(f"Calculating vocabulary for month {month}...")
    while corpus:
        for word in corpus.popitem()[1]:
            vocabulary[word] += 1

    log(f"Vocabulary for month {month} calculated.")
    return dict(vocabulary)


if not os.path.isfile(vocabularies_file_path):
    log("Calculating vocabulary counts...")
    with mp.Pool(min(os.cpu_count() or 8, 3)) as pool:
        vocabularies = pool.map(load_vocabulary, range(1, 13))
    save_pickle(vocabularies, vocabularies_file_path)
else:
    log("Vocabulary counts already exist. Loading...")
    vocabularies = load_pickle(vocabularies_file_path)

In [ ]:
training_vocabulary = vocabularies[TRAINING_MONTH - 1]

past_vocabularies = list(reversed(vocabularies[: TRAINING_MONTH - 1]))
future_vocabularies = list(reversed(vocabularies[TRAINING_MONTH:]))

testing_vocabulary_counts: T.DefaultDict[str, T.List[int]] = cll.defaultdict(
    list
)
for index, vocabulary in enumerate(vocabularies, start=1):
    if index != TRAINING_MONTH:
        for word, count in vocabulary.items():
            testing_vocabulary_counts[word].append(count)

testing_vocabulary = {
    word: float(np.mean(counts))
    for word, counts in testing_vocabulary_counts.items()
}

In [ ]:
from scipy.special import zeta

sorted_word_counts = list(
    sorted(training_vocabulary.items(), key=lambda item: item[1], reverse=True)
)

x = list(range(len(sorted_word_counts)))
y = [count for _, count in sorted_word_counts]

sorted_word_counts = list(
    sorted(testing_vocabulary.items(), key=lambda item: item[1], reverse=True)
)

test_x = list(range(len(sorted_word_counts)))
test_y = [count for _, count in sorted_word_counts]

a = 1.78
zipf_y = sum(y) * (np.array(x) ** -a) / zeta(a)

plt.clf()
ax = plt.gca()
ax.loglog(
    x,
    y,
    label="Word Frequency Distribution of Training Corpus Vocabulary",
    color="blue",
)
ax.loglog(
    test_x,
    test_y,
    label="Word Frequency Distribution of "
    "All Testing Corpus Vocabularies (Mean)",
    color="green",
)
ax.loglog(
    x,
    zipf_y,
    label=f"Zipf Distribution (a = {a:.2f})",
    color="red",
)
plt.xlim(1, len(x) * 7.75)
plt.xlabel("Word Rank (log)")
plt.ylabel("Word Frequency/Occurrence (log)")
plt.legend(loc="best")
plt.tight_layout()
plt.gcf().set_size_inches(10, 4)
plt.savefig(os.path.join(ARTEFACTS_PATH, "figure.word_frequencies.pdf"))
plt.show()

In [ ]:
def out_of_word_count(
    internal_vocabulary: T.Dict[str, int],
    external_vocabulary: T.Dict[str, int],
):
    return {
        word: count
        for word, count in external_vocabulary.items()
        if word not in internal_vocabulary
    }


print("Month, Total Words, Total Count, OOV Words, OOV Count")
for month, vocabulary in enumerate(vocabularies, start=1):
    oov_words = out_of_word_count(training_vocabulary, vocabulary)
    print(
        month,
        len(vocabulary),
        sum(vocabulary.values()),
        len(oov_words),
        sum(oov_words.values()),
        sep=", ",
    )

In [ ]:
def jaccard_similarity(vocabulary_1, vocabulary_2):
    intersection = len(
        list(frozenset(vocabulary_1).intersection(vocabulary_2))
    )
    union = (len(vocabulary_1) + len(vocabulary_2)) - intersection
    return float(intersection) / union


jaccard_similarities = [
    jaccard_similarity(*vocabulary_pair)
    for vocabulary_pair in it.product(vocabularies, vocabularies)
]

print(
    f"Jaccard similarity mean: {np.mean(jaccard_similarities):.3f} "
    f"+- {np.std(jaccard_similarities):.3f}"
)

In [ ]:
def get_labels(
    month: T.Union[str, int]
) -> T.Dict[str, T.Dict[T.Optional[str], int]]:
    return load_pickle(
        os.path.join(ARTEFACTS_PATH, f"{month}.standard.labels.pickle.gz")
    )


month_labels = [get_labels(month) for month in range(1, 13)]

In [ ]:
training_score = scores[TRAINING_MONTH - 1]
past_scores = list(reversed(scores[: TRAINING_MONTH - 1]))
future_scores = scores[TRAINING_MONTH:]


def count_predictions(scores):
    correct_predictions: T.List[T.List[int]] = []
    incorrect_predictions: T.List[T.List[int]] = []
    for score in scores:
        correct = []
        incorrect = []
        for class_score in score["Classes"].values():
            correct.append(
                int(class_score["True Positives"])
                + int(class_score["True Negatives"])
            )
            incorrect.append(
                int(class_score["False Positives"])
                + int(class_score["False Negatives"])
            )
        correct_predictions.append(correct)
        incorrect_predictions.append(incorrect)

    return correct_predictions, incorrect_predictions


training_score_correct, training_score_incorrect = count_predictions(
    [training_score]
)
training_score_correct, training_score_incorrect = (
    training_score_correct[0],
    training_score_incorrect[0],
)

past_scores_correct, past_scores_incorrect = count_predictions(past_scores)
future_scores_correct, future_scores_incorrect = count_predictions(
    future_scores
)

In [ ]:
plt.clf()

plt.yscale("log")
plt.xlabel("Month Distance from Training Month")
plt.ylabel("Number of Correct and Incorrect Predictions (log)")

plt.plot(
    list(range(1, len(past_scores_correct) + 1)),
    [sum(correct) for correct in past_scores_correct],
    "-o",
    label="Correct Past Predictions",
    color="b",
)
plt.plot(
    list(range(1, len(future_scores_correct) + 1)),
    [sum(correct) for correct in future_scores_correct],
    "-o",
    label="Correct Future Predictions",
    color="g",
)
plt.plot(
    list(range(1, len(past_scores_incorrect) + 1)),
    [sum(incorrect) for incorrect in past_scores_incorrect],
    "-o",
    label="Incorrect Past Predictions",
    color="r",
)
plt.plot(
    list(range(1, len(future_scores_incorrect) + 1)),
    [sum(incorrect) for incorrect in future_scores_incorrect],
    "-o",
    label="Incorrect Future Predictions",
    color="orange",
)

plt.legend(loc="center")
plt.tight_layout()
plt.gcf().set_size_inches(5.5, 4)
plt.savefig(
    os.path.join(
        ARTEFACTS_PATH, "figure.longitudinal_analysis_predictions.pdf"
    )
)
plt.show()

In [ ]:
from matplotlib.ticker import FuncFormatter, MultipleLocator

x_time = list(sorted(len(labels) for labels in month_labels))
y_time = [
    score["End Time"].timestamp() - score["Start Time"].timestamp()
    for score, _ in sorted(zip(scores, month_labels), key=lambda x: len(x[1]))
]
month_order = [
    month
    for month, _ in sorted(
        enumerate(month_labels, start=1), key=lambda x: len(x[1])
    )
]

plt.clf()
plt.gca().yaxis.set_major_formatter(
    FuncFormatter(lambda x, _: str(dt.timedelta(seconds=x))[:-3])
)
plt.gca().yaxis.set_major_locator(MultipleLocator(base=3600 // 4))
plt.gca().xaxis.set_major_formatter(
    FuncFormatter(lambda x, _: f"{x / 1_000_000}M")
)
plt.xlabel("Number of Documents Representing Potential Victims (millions)")
plt.ylabel("Time Taken to Test Month (H:MM)")
plt.gca().set_xticks(list(range(0, 10_000_000, 500_000)))
plt.xlim(2_500_000, 6_000_000)
for month, x, y in sorted(
    zip(month_order, x_time, y_time), key=lambda x: x[0]
):
    plt.scatter(x, y, label=str(month))
z = np.polyfit(x_time, y_time, 1)
p = np.poly1d(z)
plt.plot(x_time, p(x_time), "--", color="black", label="Trend Line")
plt.legend(title="Months", ncol=2, loc="best")
plt.tight_layout()
plt.gcf().set_size_inches(5.5, 4)
plt.savefig(
    os.path.join(ARTEFACTS_PATH, "figure.longitudinal_analysis_time.pdf")
)
plt.show()

In [ ]:
print(
    "Time taken for testing:",
    (scores[-1]["End Time"] - scores[0]["Start Time"]) / MAX_CONCURRENT_MONTHS,
)